In [ ]:
%matplotlib inline

Unstructured grid
=================

The interpolation of this object is based on a
[R*Tree](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.RTree.html#pyinterp.RTree)
structure. To begin with, we start by building this object. By default, this
object considers the WGS-84 geodetic coordinate system. But you can define
another one using the class
[System](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.geodetic.System.html#pyinterp.geodetic.System).

In [ ]:
import matplotlib.pyplot
import numpy
import pyinterp

mesh = pyinterp.RTree()

Then, we will insert points into the tree. The class allows you to add points
using two algorithms. The first one, called
[packing](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.RTree.packing.html#pyinterp.RTree.packing),
will enable you to enter the values in the tree at once. This mechanism is the
recommended solution to create an optimized in-memory structure, both in terms
of construction time and queries. When this is not possible, you can insert new
information into the tree as you go along using the
[insert](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.RTree.packing.html#pyinterp.RTree.insert)
method.

In [ ]:
SIZE = 2000
X0, X1 = 80, 170
Y0, Y1 = -45, 30
lons = numpy.random.uniform(low=X0, high=X1, size=(SIZE, ))
lats = numpy.random.uniform(low=Y0, high=Y1, size=(SIZE, ))
data = numpy.random.random(size=(SIZE, ))

Populates the search tree


In [ ]:
mesh.packing(numpy.vstack((lons, lats)).T, data)

When the tree is created, you can interpolate data with two algorithms:

-   [Inverse Distance Weighting](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.RTree.inverse_distance_weighting.html#pyinterp.RTree.inverse_distance_weighting)
    or IDW
-   [Radial Basis Function](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.RTree.radial_basis_function.html#pyinterp.RTree.radial_basis_function)
    or RBF

Yon can also search the
[nearest neighbors](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.RTree.query.html#pyinterp.RTree.query)
on the tree.

---
**Note**

When comparing an RBF to IDW, IDW will never predict values higher than
the maximum measured value or lower than the minimum measured value.
However, RBFs can predict values higher than the maximum values and
lower than the minimum measured values.

---

In this example, we will under-sample the source grid at 1/32 degree
over an area of the globe.

In [ ]:
STEP = 1 / 32
mx, my = numpy.meshgrid(numpy.arange(X0, X1 + STEP, STEP),
                        numpy.arange(Y0, Y1 + STEP, STEP),
                        indexing="ij")

idw, neighbors = mesh.inverse_distance_weighting(
    numpy.vstack((mx.flatten(), my.flatten())).T,
    within=False,  # Extrapolation is forbidden
    k=11,  # We are looking for at most 11 neighbours
    num_threads=0)
idw = idw.reshape(mx.shape)

The with the RBF method


In [ ]:
rbf, neighbors = mesh.radial_basis_function(
    numpy.vstack((mx.flatten(), my.flatten())).T,
    within=False,  # Extrapolation is forbidden
    k=11,  # We are looking for at most 11 neighbours
    rbf="linear",
    num_threads=0)
rbf = rbf.reshape(mx.shape)

Let's visualize our interpolated data


In [ ]:
fig = matplotlib.pyplot.figure(figsize=(10, 8))
fig.patch.set_alpha(0.0)
ax1 = fig.add_subplot(211)
pcm = ax1.pcolormesh(mx, my, idw, cmap='jet', shading='auto', vmin=0, vmax=1)
ax1.set_title("IDW interpolation")
ax2 = fig.add_subplot(212)
pcm = ax2.pcolormesh(mx, my, rbf, cmap='jet', shading='auto', vmin=0, vmax=1)
ax2.set_title("RBF interpolation")
fig.colorbar(pcm, ax=[ax1, ax2], shrink=0.8)
fig.show()